## 신경망 모델 구성하기
- 대체로 torch.nn에 어지간한거 모두 다 있다. torch의 모든 모듈은 nn.Module의 하위 클래스로 신경망은 여러 layer로 구성된 모듈이다.
- 아래의 예제를 보면서 이해해보자

In [37]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda

In [38]:
training_data = datasets.FashionMNIST(
    root = "/raid/jskim/data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "/raid/jskim/data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [39]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [41]:
print(train_dataloader)

## 클래스로 정의하기
- 신경망 모델을 커다란 하나의 torch.nn.Module 모듈이라 생각하고 만들면 좋다.

In [42]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [43]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [44]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred  = pred_probab.argmax(1)

In [45]:
print(logits)
print(pred_probab)
print(y_pred)

tensor([[0.0000, 0.0000, 0.1498, 0.0000, 0.0000, 0.0145, 0.0000, 0.0957, 0.0000,
         0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0973, 0.0973, 0.1130, 0.0973, 0.0973, 0.0987, 0.0973, 0.1071, 0.0973,
         0.0973]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([2], device='cuda:0')


In [46]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0173,  0.0277,  0.0138,  ..., -0.0171, -0.0296,  0.0137],
        [ 0.0328,  0.0309,  0.0066,  ...,  0.0320,  0.0254,  0.0251]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0179, 0.0196], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0205,  0.0008,  0.0394,  ..., -0.0281, -0.0010,  0.0294],
        [ 0.0252, -0.0190,  0.0364,  ..., -0.0422,  0.0294, -0.0349]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([ 5.9048e-05, -1.5337e-02], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0437,  0.0023, -0.0342,  ...,  0.0391, -0.0431, -0.0400],
        [ 0.0068, -0.0192,  0.017

## 하이퍼 파라미터
- 모델 최적화 과정을 제어할 수 있는 매개변수로 ephoch, batch size, learning rate가 기본적이다

In [47]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [48]:
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [49]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # 요거 tutorial에 안되있어서 찾아다녔네
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_Batchs = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()
            
    test_loss /= batch_size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [50]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302393  [    0/60000]
loss: 2.304411  [ 6400/60000]
loss: 2.292821  [12800/60000]
loss: 2.286015  [19200/60000]
loss: 2.279091  [25600/60000]
loss: 2.259528  [32000/60000]
loss: 2.258271  [38400/60000]
loss: 2.242497  [44800/60000]
loss: 2.249248  [51200/60000]
loss: 2.212293  [57600/60000]
Test Error: 
 Accuracy: 39.0%, Avg loss: 5.477993 

Epoch 2
-------------------------------
loss: 2.247454  [    0/60000]
loss: 2.257750  [ 6400/60000]
loss: 2.220712  [12800/60000]
loss: 2.213846  [19200/60000]
loss: 2.189607  [25600/60000]
loss: 2.157643  [32000/60000]
loss: 2.163785  [38400/60000]
loss: 2.134794  [44800/60000]
loss: 2.160106  [51200/60000]
loss: 2.061599  [57600/60000]
Test Error: 
 Accuracy: 40.1%, Avg loss: 5.197614 

Epoch 3
-------------------------------
loss: 2.155849  [    0/60000]
loss: 2.177218  [ 6400/60000]
loss: 2.100712  [12800/60000]
loss: 2.092805  [19200/60000]
loss: 2.030548  [25600/60000]
loss: 1.989696  [32000/600